In [16]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")

### Pre-processing

In [17]:
df_109 = pd.read_csv(("../Resources/109justice.csv"))
df_109.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
2027,109,2017,2,0,1,2,55
2028,109,2017,1,0,9,9,0
2029,109,2017,2,0,2,2,0
2030,109,2017,2,0,2,6,0
2031,109,2017,1,0,1,6,0


In [3]:
# df_109["direction"] = np.where(df_106["direction"] == 1, 0, 1)
# df_109.tail(5)
# # df_109.count()

In [18]:
target_109 = df_109["direction"]
target_names = ["conservative", "liberal"]

In [19]:
data_109 = df_109.drop(["direction", "justice"], axis=1)
feature_names = data_109.columns
data_109.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,1993,0,2,4,0
1,1993,0,2,4,0
2,1993,0,2,3,0
3,1993,0,2,3,0
4,1993,0,9,4,0


In [20]:
X_scaler = StandardScaler().fit(data_109)
X_scaled = X_scaler.transform(data_109)

X = X_scaled
y = target_109

### Evaluating different classifiers

In [21]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators=300)
clf3 = GaussianNB()
clf4 = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf1, clf2, clf3, clf4], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))


5-fold cross validation:

F1: 0.068 [Logistic Regression]
F1: 0.310 [Random Forest]
F1: 0.494 [Naive Bayes]
F1: 0.000 [Linear SVM]


In [25]:
# taking out SVM & Logistic Reg (since scores were low) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[1], clf2),
                                            (labels[2], clf3)],
                                            voting = 'hard')

In [26]:
v_clf_soft = VotingClassifier(estimators = [(labels[1], clf2),
                                            (labels[2], clf3)],
                                            voting = 'soft')

In [27]:
new_labels = ["Random Forest", "Naive Bayes", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf2, clf3, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.310 [Random Forest]
F1: 0.494 [Naive Bayes]
F1: 0.494 [Voting_Classifier_Hard]
F1: 0.320 [Voting_Classifier_Soft]


### Training the Naive Bayes model

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [30]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [31]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.533
Test Acc: 0.530


In [32]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.44      0.64      0.52       205
     liberal       0.65      0.45      0.53       303

   micro avg       0.53      0.53      0.53       508
   macro avg       0.55      0.55      0.53       508
weighted avg       0.57      0.53      0.53       508



In [33]:
confusion_matrix(predictions, y_test)
predictions[:10], y_test[:10].ravel()

(array([1, 2, 1, 2, 2, 1, 1, 1, 2, 2], dtype=int64),
 array([2, 1, 1, 2, 2, 2, 1, 2, 2, 2], dtype=int64))